### FINAL PROJECT MACHINE LEARNING

In [1]:
import pandas as pd
import numpy as np
import re

## 1st part: Preprocessing data

#### 1. CLEANING COLUMN 'NAME'

In [2]:

#load primary file from clients 
excel_file_path = 'ingredient_orginal.xlsx'  
df = pd.read_excel(excel_file_path)

# Create a new DataFrame
data = pd.DataFrame()

# Specify the number of rows to split (in this case, the first 3 rows)
num_rows_to_split = 3

# Iterate through each row in the original DataFrame
for index, row in df.iterrows():
    # Split the 'Name' column only for the first 3 rows
    if index < num_rows_to_split:
        names = str(row['Name']).split('/')  
        num_names = len(names)

        # Duplicate the row for each name
        duplicated_row = pd.DataFrame([row] * num_names)

        # Update the "Names" column with individual names
        duplicated_row['Name'] = names

    
        data = pd.concat([data, duplicated_row], ignore_index=True)
    else:
        
        data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)



#### 2. Cleaning column CASCODE

In [3]:
data['CASCODE'].value_counts()

-                                       367
25133-97-5 / 25035-69-2 / 25212-88-8     18
84696-07-01                               8
9016-45-9                                 6
90028-76-5                                6
                                       ... 
141714-54-7                               1
68555-87-3                                1
92502-55-1                                1
103458-51-1                               1
87435-35-6                                1
Name: CASCODE, Length: 5193, dtype: int64

In [ ]:
data['CASCODE'] = data['CASCODE'].astype(str)

#Extract only CASCODE without other irelevant data
data['CASCODE'] = data['CASCODE'].str.extract(r'^([\d\-]+)')

In [ ]:
#Empty cels with '-'
data['CASCODE'] = data['CASCODE'].replace({'-': np.nan})

In [ ]:
data['CASCODE'].value_counts()

In [ ]:
data.info()

In [ ]:
data['CASCODE'].value_counts()

In [ ]:
# Save in Excel file after preprocesing
data.to_excel('C:\\Users\\Administrator\\Desktop\\FINALEN PROEKT\\preprocesed_ingredients.xlsx', index=False)


## 2nd part: Web scraping

In [ ]:
#load the dataset that we extraced after preprocesing
excel = pd.read_excel('preprocesed_ingredients.xlsx')
data = pd.DataFrame(excel)

data['substitute_names'] = ''

In [ ]:

from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize Chrome WebDriver
driver = webdriver.Chrome()

    #Iterate thru names  
for index, row in data.iterrows():
    substance_name = row['Name']

    # Check if CAS code is already present, if so, continue to the next row
    if pd.notnull(row['CASCODE']):
        continue

    # Construct the search URL for the substance
    search_url = f"https://commonchemistry.cas.org/results?q={substance_name.replace(' ', '%20')}"

    # Open the URL in the Chrome WebDriver
    driver.get(search_url)

    # Wait for the CAS number element to be present
    wait = WebDriverWait(driver, 6)
    cas_element = None

    try:
        cas_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="result-rn"]')))
        cas_number = cas_element.text.strip()
        data.at[index, 'CASCODE'] = cas_number
    except:
        data.at[index, 'CASCODE'] = 'no cascode found'

# Quit the WebDriver 
driver.quit()

In [ ]:
#Save the new excel file after scraping for CAS numbers on Common chemistry website
data.to_excel('C:\\Users\\Administrator\\Desktop\\FINALEN PROEKT\\ingredients_with_name_scraping.xlsx', index=False)

In [ ]:
#load the dataset that we extraced after scraping by name for CAS numbers
excel = pd.read_excel('ingredients_with_name_scraping.xlsx')
data = pd.DataFrame(excel)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 3)

for cas_code in data['CASCODE']:
    try:
        driver.get(f"https://commonchemistry.cas.org/detail?cas_rn={cas_code}")

        try:
            element = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@_ngcontent-serverapp-c40=""]')))
            element.click()

            dynamic_content = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.detail-synonym-box')))
            extracted_info = dynamic_content.text

            # Append the extracted information to the 'substitute_names' column for the corresponding CAS code
            data.loc[data['CASCODE'] == cas_code, 'substitute_names'] = extracted_info

        except TimeoutException:
            try:
                dynamic_content = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.detail-synonym-box')))
                extracted_info = dynamic_content.text

                # Append the extracted information to the 'substitute_names' column for the corresponding CAS code
                data.loc[data['CASCODE'] == cas_code, 'substitute_names'] = extracted_info

            except TimeoutException:
                # If neither element nor data found, mark as "Non-existent CAS code"
                data.loc[data['CASCODE'] == cas_code, 'substitute_names'] = "Non-existent CAS code"

    except TimeoutException:
        # If URL not accessible, mark as "URL not accessible"
        data.loc[data['CASCODE'] == cas_code, 'substitute_names'] = "URL not accessible"

driver.quit()

In [ ]:
#Save the new excel file after scraping for CAS numbers and substitute names on Common chemistry website
data.to_excel('C:\\Users\\Administrator\\Desktop\\FINALEN PROEKT\\ingredients_after_all_scraping.xlsx', index=False)